In [0]:
from settings import create_spark_session
from pyspark.sql.functions import col, sum as _sum, max as _max, lit
import logging

In [0]:
def save_table(df, table_name, gold_schema, partition_col):
    df.write.format("delta") \
        .mode("append") \
        .partitionBy(partition_col) \
        .saveAsTable(f"{gold_schema}.{table_name}")

def build_fact_sales(logger, spark, silver_schema, gold_schema, ingest_date):
    try:
        logger.info("Lendo dados da Silver: orders_cleaned")
        orders = spark.table(f"{silver_schema}.orders_cleaned") \
            .filter(col("ingest_date") == ingest_date)
        
        logger.info("Lendo dados da Silver: order_items_cleaned")
        order_items = spark.table(f"{silver_schema}.order_items_cleaned") \
            .filter(col("ingest_date") == ingest_date)


        logger.info("Lendo dados da Silver: inventory_cleaned")
        inventory = spark.table(f"{silver_schema}.inventory_cleaned")   

        logger.info("Construindo tabela de fato: fact_sales")
        fact_sales = orders.join(order_items, on="order_id", how="inner") \
            .select(
                "order_id", "customer_id", "product_id", "order_date",
                "quantity", "unit_price", "total_price", "status"
            ) \
            .withColumn("ingest_date", lit(ingest_date))

        logger.info("Escrevendo fact_sales na Gold com particionamento por ingest_date")
        save_table(fact_sales, "fact_sales", gold_schema, "ingest_date")

    except Exception as e:
        logger.error(f"Erro ao construir fact_sales: {e}")

def build_dim_customers(logger, spark, silver_schema, gold_schema, ingest_date):
    try:
        logger.info("Lendo dados da Silver: customers_cleaned")
        customers = spark.table(f"{silver_schema}.customers_cleaned")

        logger.info("Construindo dimensão: dim_customers")
        dim_customers = customers.select(
            "customer_id", "name", "email", "address", "created_at"
        ).dropDuplicates(["customer_id"]) \
         .withColumn("ingest_date", lit(ingest_date))

        logger.info("Escrevendo dim_customers na Gold com particionamento por ingest_date")
        save_table(dim_customers, "dim_customers", gold_schema, "ingest_date")

    except Exception as e:
        logger.error(f"Erro ao construir dim_customers: {e}")

def build_dim_products(logger, spark, silver_schema, gold_schema, ingest_date):
    try:
        logger.info("Lendo dados da Silver: products_cleaned")
        products = spark.table(f"{silver_schema}.products_cleaned")

        logger.info("Construindo dimensão: dim_products")
        dim_products = products.select(
            "product_id", "name", "category", "price"
        ).dropDuplicates(["product_id"]) \
         .withColumn("ingest_date", lit(ingest_date))

        logger.info("Escrevendo dim_products na Gold com particionamento por ingest_date")
        save_table(dim_products, "dim_products", gold_schema, "ingest_date")

    except Exception as e:
        logger.error(f"Erro ao construir dim_products: {e}")

def build_current_inventory(logger, spark, silver_schema, gold_schema, ingest_date):
    try:
        logger.info("Lendo dados da Silver: inventory_cleaned")
        inventory = spark.table(f"{silver_schema}.inventory_cleaned")

        logger.info("Construindo tabela agregada: current_inventory")
        current_inventory = inventory.groupBy("product_id") \
            .agg(
                _sum("change").alias("stock_quantity"),
                _max("timestamp").alias("last_updated")
            ) \
            .withColumn("ingest_date", lit(ingest_date))

        logger.info("Escrevendo current_inventory na Gold com particionamento por ingest_date")
        save_table(current_inventory, "current_inventory", gold_schema, "ingest_date")

    except Exception as e:
        logger.error(f"Erro ao construir current_inventory: {e}")

def main(spark, silver_path, gold_schema, ingest_date):
    logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
    logger = logging.getLogger("GoldLayer")

    # Cria o schema Gold se não existir
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {gold_schema}")

    logger.info("Iniciando processamento da camada Gold")

    build_fact_sales(logger, spark, silver_schema, gold_schema, ingest_date)
    build_dim_customers(logger, spark, silver_schema, gold_schema, ingest_date)
    build_dim_products(logger, spark, silver_schema, gold_schema, ingest_date)
    build_current_inventory(logger, spark, silver_schema, gold_schema, ingest_date)

    logger.info("Camada Gold finalizada com sucesso")

In [0]:
gold_config = {
    "spark.sql.shuffle.partitions": "300",
    "spark.default.parallelism": "300",
    "spark.sql.adaptive.enabled": "true",
    "spark.sql.adaptive.skewJoin.enabled": "true",
    "spark.databricks.delta.optimizeWrite.enabled": "true",
    "spark.databricks.delta.autoCompact.enabled": "true",
    "spark.sql.autoBroadcastJoinThreshold": "104857600",
    "spark.sql.parquet.filterPushdown": "true"
}

app_name = "GoldLayer"
spark = create_spark_session(app_name, gold_config)

Explicação das configs para a camada Gold
  - **spark.sql.shuffle.partitions = 300**
    - A camada Gold geralmente tem dados agregados, otimizados e prontos para consumo.
    - É comum que aqui as operações de shuffle (como joins, agregações) sejam maiores, então aumentar o número de partições para 300 ajuda a distribuir melhor a carga, evitando partições muito grandes que causam gargalos.

  - **spark.default.parallelism = 300**
    - Controla o paralelismo padrão para operações RDD e shuffle.
    - Mantendo esse valor alto ajuda a aproveitar melhor os recursos do cluster, principalmente em operações batch de alta escala.

  - **spark.sql.adaptive.enabled = true**
    - Ativa a execução adaptativa de queries (Adaptive Query Execution - AQE).

    - O Spark pode otimizar dinamicamente planos de execução durante o runtime, ajustando a quantidade de shuffle, reduzindo skew e melhorando desempenho.

  - **spark.sql.adaptive.skewJoin.enabled = true**
    - Específico para mitigar data skew em joins.
    - Na camada Gold, onde joins entre tabelas dimensionais e fatos são comuns, isso ajuda a evitar que partições muito desbalanceadas causem lentidão.

  - **spark.databricks.delta.optimizeWrite.enabled = true**
    - Otimiza a escrita dos arquivos Delta, criando arquivos maiores e menos fragmentados.
    - Importante para a camada Gold, para melhorar a performance de leitura e compactação.

  - **spark.databricks.delta.autoCompact.enabled = true**
    - Ativa a compactação automática de arquivos pequenos após escrita, evitando muitos pequenos arquivos.
    - Pequenos arquivos impactam a performance de leitura; isso mantém o Delta otimizado.

  - **spark.sql.autoBroadcastJoinThreshold = 104857600 (100 MB)**
    - Define o limite máximo para broadcast join automático.
    - Broadcast join é eficiente para joins onde uma tabela é pequena o suficiente para ser replicada em cada executor.
    - Um valor maior (100MB) permite mais casos de broadcast join, acelerando a execução de joins na camada Gold.

  - **spark.sql.parquet.filterPushdown = true**
    - Permite que filtros sejam aplicados diretamente na leitura de arquivos Parquet (e Delta).
    - Reduz a quantidade de dados carregados em memória, acelerando consultas.

Por que aplicar essas configs só na camada Gold?
  - Camada Gold é a mais refinada e consumida por BI/analytics, então a prioridade é performance e otimização para consultas rápidas.
  - As operações geralmente envolvem joins complexos, agregações e leituras frequentes.
  - Essas configs ajudam o Spark a otimizar shuffle, paralelismo, joins e escrita, melhorando latência e throughput.


In [0]:
# Define os caminhos base das camadas silver e Gold
silver_schema = "lakehouse.a_silver"
gold_schema = "lakehouse.a_gold"

# Define a data de ingestão a ser processada (ex: '2025-06-12')
ingest_date = "2025-06-15"

# Chama a função principal da camada Silver
main(spark, silver_schema, gold_schema, ingest_date)


2025-06-15 15:50:09,214 - INFO - Iniciando processamento da camada Gold
2025-06-15 15:50:09,214 - INFO - Lendo dados da Silver: orders_cleaned
2025-06-15 15:50:09,215 - INFO - Lendo dados da Silver: order_items_cleaned
2025-06-15 15:50:09,216 - INFO - Lendo dados da Silver: inventory_cleaned
2025-06-15 15:50:09,217 - INFO - Construindo tabela de fato: fact_sales
2025-06-15 15:50:09,217 - INFO - Escrevendo fact_sales na Gold com particionamento por ingest_date
2025-06-15 15:50:15,677 - INFO - Lendo dados da Silver: customers_cleaned
2025-06-15 15:50:15,678 - INFO - Construindo dimensão: dim_customers
2025-06-15 15:50:15,679 - INFO - Escrevendo dim_customers na Gold com particionamento por ingest_date
2025-06-15 15:50:19,954 - INFO - Lendo dados da Silver: products_cleaned
2025-06-15 15:50:19,955 - INFO - Construindo dimensão: dim_products
2025-06-15 15:50:19,959 - INFO - Escrevendo dim_products na Gold com particionamento por ingest_date
2025-06-15 15:50:23,284 - INFO - Lendo dados da S